In [41]:
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
import time
import pandas as pd
import re
from tqdm.auto import tqdm

In [3]:
# Função para determinar se houve alagamento ou não em um determinado dia
def houve_alag(soup):
    # Texto padrão quando não houve alagamentos
    texto = 'Não há registros de alagamentos para essa data.'

    # Variável para alagamento. 0, sem alagamento. 1, com alagamento
    flag_alag = 0

    alag = soup.find_all(string = re.compile('registros'))
    
    if (alag == []):
        flag_alag = 1
        return flag_alag
    else:
        flag_alag = 0
        return flag_alag

In [92]:
# Cria dataframe para receber o conteúdo
Campos = ['Data',
           'Alagamento',
           'Bairro',
           'Tipo',
           'Horário Inicial',
           'Horário Final',
           'Endereço',
           'Sentido',
           'Referência'
           ]
df = pd.DataFrame(columns=Campos)

# Montagem dinâmica das datas a serem varridas:
start = date(2017, 1, 1)
end = date(2017, 1, 17)

# Datas com problemas
# 13/01/2019
# 15/11/2019
# 22/06/2021
# 29/10/2022
# 12/03/2023
# 17/01/2017

# Quando a função estiver totalmente funcional
# end = date.today()
data_pesquisada = start
dia = timedelta(days=1)

index= 0

# Inicia o preenchimento do dataframe
while data_pesquisada < end:
    data_formatada = data_pesquisada.strftime('%d')+'%2F'+data_pesquisada.strftime('%m')+'%2F'+data_pesquisada.strftime('%Y')
    URL_pesquisada = 'https://www.cgesp.org/v3/alagamentos.jsp?dataBusca='+data_formatada+'&enviaBusca=Buscar'
    #print(URL_pesquisada)

    # Faz a requisição para a página
    try:
        response = requests.get(URL_pesquisada)
    except requests.exceptions.Timeout:
        print('Timeout informado pelo sistema')
    except requests.exceptions.TooManyRedirects:
        print('Problema na formação da URL. Favor tentar com outra')
    except requests.exceptions.RequestException as e:
        print('Erro catastrófico')
        raise SystemExit(e)

    # Analisa o HTML da página com o BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
        
    if (houve_alag(soup)==0):
        #print(f"Em {data_pesquisada} não houve registros de alagamentos.")
        df.at[index,'Data'] =  data_pesquisada
        df.at[index,'Alagamento'] = 0
        df.at[index,'Bairro'] = 0
        df.at[index,'Tipo'] = 0
        df.at[index,'Horário Inicial'] = 0
        df.at[index,'Horário Final'] = 0
        df.at[index,'Endereço'] = 0
        df.at[index,'Sentido'] = 0
        df.at[index,'Referência'] = 0
        index = index +1 
    else:
        index_linhas = 0
        for pontos in tqdm(soup.find_all('td', {'class':'total-pts arial-bairros-alag'}), desc='Pontos de Alagamento'):
            texto = pontos.string
            no_alag = int(texto[0:texto.find('pt')-1])
            #print(f'Data: {data_pesquisada}, No. alagamentos: {no_alag}')

            linhas = soup.find_all('div',{'class':'ponto-de-alagamento'})
            

            # Vamos iterar sobre a quantidade de pontos de alagamento
            for pt_alag in range(no_alag):
            
                df.at[index,'Data'] =  data_pesquisada
                df.at[index,'Alagamento'] = 1

                pai = pontos.parent
            
                df.at[index,'Bairro'] = pai.find('td',{'class':'bairro arial-bairros-alag linha-pontilhada'}).text.strip()
                df.at[index,'Tipo'] = linhas[index_linhas].find('li')['title']
            
                #'Horário Inicial',
                texto1 = linhas[index_linhas].find('li', {'class':'arial-descr-alag col-local'}).text
                horas =re.findall('(?:(\d\d?):)?([0-5][0-9])',texto1)
                df.at[index,'Horário Inicial'] = str(horas[0][0])+':'+str(horas[0][1])
                
                #'Horário Final'
                df.at[index,'Horário Final'] = str(horas[1][0])+':'+str(horas[1][1])

                # Endereço
                df.at[index,'Endereço'] = texto1[texto1.find(horas[1][1])+2:len(texto1)]

                #'Sentido',
                texto2 = linhas[index_linhas].find_all('li', {'class':'arial-descr-alag'})
                info = texto2[1].get_text()
                df.at[index,'Sentido'] = info[8:info.find('Ref')]

                #'Referência'
                df.at[index,'Referência'] = info[info.find('Ref')+11:len(info)]
            
                index = index +1 
                index_linhas = index_linhas +1
    data_pesquisada = data_pesquisada + dia
    time.sleep(1)
df

Pontos de Alagamento:   0%|          | 0/7 [00:00<?, ?it/s]

Pontos de Alagamento:   0%|          | 0/2 [00:00<?, ?it/s]

Pontos de Alagamento:   0%|          | 0/9 [00:00<?, ?it/s]

Pontos de Alagamento:   0%|          | 0/5 [00:00<?, ?it/s]

Pontos de Alagamento:   0%|          | 0/7 [00:00<?, ?it/s]

...

Pontos de Alagamento:   0%|          | 0/2 [00:00<?, ?it/s]

Pontos de Alagamento:   0%|          | 0/1 [00:00<?, ?it/s]

Pontos de Alagamento:   0%|          | 0/7 [00:00<?, ?it/s]

Pontos de Alagamento:   0%|          | 0/13 [00:00<?, ?it/s]

,Data,Alagamento,Bairro,Tipo,Horário Inicial,Horário Final,Endereço,Sentido,Referência
0,2017-01-01,0,0,0,0,0,0,0,0
1,2017-01-02,1,Vila Maria/ Vila Guilherme,Inativo Transitável,18:27,19:16,MARGINAL TIETE ASCB EXPRESSA,A SENNA/CASTELO B,PTE DA VILA GUILHERME
2,2017-01-02,1,Vila Maria/ Vila Guilherme,Inativo Transitável,18:36,19:01,AV MORVAN DIAS DE FIGUEIREDO,A SENNA/CASTELO B,R DA COROA
3,2017-01-02,1,Itaquera,Inativo Transitável,18:43,19:18,:43 a 19:18AV JACU-PESSEGO N TRABALHADORES,MAUA/A SENNA,R MONTANHAS
4,2017-01-02,1,Mooca,Inativo Transitável,18:56,19:01,"AV CELSO GARCIA,",CENTRO/BAIRRO,AV ALVARO RAMOS
...,...,...,...,...,...,...,...,...,...
117,2017-01-16,1,Santo Amaro,Inativo Intransitável,22:51,23:00,AV DAS NACOES UNIDAS,INTERLAGOS/CASTELO,R PROF CAMPOS DE OLIVEIRA
118,2017-01-16,1,Santo Amaro,Inativo Transitável,22:53,02:48,AV VITOR MANZINI,CENTRO/BAIRRO,ALTURA DO NÚMERO 260
119,2017-01-16,1,Santo Amaro,Inativo Intransitável,23:33,01:19,AV STO AMARO,BAIRRO/CENTRO - ÚNICA,AV ROQUE PETRONI JUNIOR
120,2017-01-16,1,Ipiranga,Inativo Intransitável,23:40,00:46,"AV PROF ABRAAO DE MORAIS,",ÚNICA,R RIBEIRO LACERDA


In [91]:
data_pesquisada

datetime.date(2017, 1, 17)

In [94]:
df.to_csv('Alagamentos_SP.csv', encoding='utf-8-sig')